In [0]:
df = spark.read.format("parquet")\
    .load("abfss://bronze@sadec25etlproject.dfs.core.windows.net/customers")

In [0]:
df.display()

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.functions import split
df = df.withColumn("domains", split(col('email'), '@')[1])
df.display()

In [0]:
from pyspark.sql.functions import count

df.groupBy("domains").agg(count("customer_id").alias("total_customers")).display()

In [0]:
df.groupBy("domains").agg(count("customer_id").alias("total_customers")).sort("total_customers", ascending=False).display()

In [0]:
import time

df_gmail = df.filter(col("domains")=="gmail.com")
#df_gmail.display()
time.sleep(5)

df_yahoo = df.filter(col("domains")=="yahoo.com")
#df_yahoo.display()
time.sleep(5)

df_hotmail = df.filter(col("domains")=="hotmail.com")
#df_hotmail.display()
time.sleep(5)

In [0]:
from pyspark.sql.functions import concat, lit, col
df = df.withColumn("full_name", concat(col("first_name"), lit(" "), col("last_name")))
df.display()

In [0]:
df = df.drop("first_name", "last_name")
df.display()

In [0]:
df = df.drop("_rescued_data")
df.display()

In [0]:
df.write.mode("overwrite").format("delta").save("abfss://silver@sadec25etlproject.dfs.core.windows.net/customers")

# instead of append using overwrite, because silverlayer is used as a transient layer don't need to store the data

In [0]:
%sql
-- create a silver schema in dec25etlproject catalog
CREATE SCHEMA IF NOT EXISTS dec25etlproject.silver;

In [0]:
%sql
-- create transformed silver customers table in silver schema by using delta table in silver sa location
CREATE TABLE IF NOT EXISTS dec25etlproject.silver.customers_silver
USING DELTA
LOCATION 'abfss://silver@sadec25etlproject.dfs.core.windows.net/customers'

In [0]:
%sql
SELECT * FROM dec25etlproject.silver.customers_silver;